### ASSIST tests: (84100) Farnocchia integrations

Here we demonstrate the integration of (84100) Farnocchia, a main belt asteroid,using ASSIST.  

ASSIST is a set of python-wrapped C functions that implement an ephemeris quality-integrator that have been compiled into a library.  That library is imported with the assist.py package.

The primary function in assist.py is 'integration_function', which integrates massless test particles in the field of the Sun, planets, Moon, and 16 massive asteroids.  It also includes the J2, J3, and J4 gravitational harmonics of the Earth, the J2 gravitational harmonic of the Sun, and the solar GR terms (using the PPN formulation).  

The positions of the massive bodies come from two binary files, both from JPL.  The first is for the Sun, planets, and Moon, with the latest DE441 ephemeris. The other is for the asteroids, corresponding to DE441.  

The coordinate frame and units are not flexible, currently.  The coordinate frame is the equatorial ICRF, which is the native coordinate system for the JPL binary files.  Note that this is equatorial rather than ecliptic.  In addition, the native coordinates are barycentric, rather than heliocentric. 

For units we use solar masses, au, and days.  The independent time coordinate is TDB in Julian days.

The underlying numerical integrator is IAS15 (Rein & Liu 2015), a 15th order predictor-corrector integrator with an adaptive step-size.  Each time step involves eight sub-steps.  We have modified the integrator to output the state at each of the sub-steps in order to support interpolation of the output.

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import ctypes
import importlib

import spiceypy as spice

import pkg_resources
import sys
sys.path.append(r'/Users/mholman/assist')
import assist

jd_ref = spice.j2000()

plt.rc('text', usetex = True)

au_km = 149597870.700

In [2]:
# This cell defines the initial conditions we have used for these integrations.
# The dynamical state currently available on Horizons might be slightly different because additional
# observations have been incorporated into the orbital solution.
#
# From Horizons
# DE441
# (84100) Farnocchia
#2458849.500000000 = A.D. 2020-Jan-01 00:00:00.0000 TDB 
row = [9.572786624350362E-01, -2.101947659454845E+00, -7.418444809938682E-01,
    9.715385994801106E-03, 6.153629531316274E-03, 1.549521077070705E-03]


In [8]:
%%time
tstart, tstep = 2458849.5-jd_ref, 20.0

instates = np.array([row])
n_var = 0
n_particles = 1
geocentric = 0

invar_part = None
invar = None

# Turn off the non-grav terms
part_params = None
var_part_params = None

nsubsteps = 10
hg = np.arange(0, 1.1, 0.1, dtype=np.double)


with open('farnocchia_sb.txt') as file:
    file.readline()
    file.readline()
    file.readline()
    for line in file:
        trange = float(line.split()[0])
        vec = np.array(line.rstrip().split()[-3:], dtype=np.double)
        tend = tstart + trange
        times, states, var, var_ng, status = assist.integration_function(tstart, tend, tstep, geocentric, 
                                                                       n_particles, instates, part_params, n_var, invar_part, invar, var_part_params, hg,
                                                                       nsubsteps=nsubsteps)
        GMsun, sun = assist.all_ephem(0, jd_ref, times[-1])
        holman_sb=vec/au_km + sun[0:3]
        print(trange, np.linalg.norm((holman_sb-states[-1][0][0:3])*au_km*1e3))


10.0 0.00014947830043743338
100.0 0.0016201784769183658
1000.0 0.014035958881276128
10000.0 2.386006594905532
100000.0 31.57706081863238
CPU times: user 8.54 s, sys: 2.22 s, total: 10.8 s
Wall time: 10.8 s


Speed tests:

length (days) time (s)
1e1           0.00312
1e2           0.00943
1e3           0.0700
1e4           0.457
1e5           4.56
1e6          57.2


Farnocchia
     10 d:      0.019
    100 d:      0.021
   1000 d:      0.020
  10000 d:      0.051
 100000 d:      0.379
1000000 d:      3.419